In [1]:
import copy
import os
import subprocess
import time
import tempfile
import sys

from pathlib import Path
from numpy.linalg import inv, norm, pinv, svd,eig
from scipy.optimize import fmin_bfgs, fmin_slsqp, minimize
from pinocchio.visualize import MeshcatVisualizer

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pinocchio as pin

In [2]:
urdf_file_path = Path("/home/regulus/ros2_ws/src/ada_ros2/scripts/articulated_fork.urdf")
model, collision_model, visual_model = pin.buildModelsFromUrdf(str(urdf_file_path), None, pin.JointModelFreeFlyer())
data = model.createData()
end_effector_joint_id = model.getJointId("roll_joint")
end_effector_frame_id = model.getFrameId("end_effector_joint")
viz = MeshcatVisualizer(model, collision_model, visual_model)
print(model)
print(list(model.idx_qs))
print(list(model.idx_vs))

Nb joints = 4 (nq=10,nv=8)
  Joint 0 universe: parent=0
  Joint 1 root_joint: parent=0
  Joint 2 pitch_joint: parent=1
  Joint 3 roll_joint: parent=2

[0, 0, 7, 8]
[0, 0, 6, 7]


In [3]:
def config_to_q(config):
    return np.array([
        0, # Base offset x
        0, # Base offset y
        0, # Base offset z
        0, # scaling
        0, # scaling
        0, # scaling
        1, # Default
        config[0], # pitch_joint
        np.cos(config[1]), # roll_joint (cos)
        np.sin(config[1]), # roll_joint (sin)
    ])

def angle_mag_to_wrench(theta, phi, magnitude):
    # Convert polar and azimuthal angles to Cartesian coordinates
    x = magnitude * np.sin(theta) * np.cos(phi)
    y = magnitude * np.sin(theta) * np.sin(phi)
    z = magnitude * np.cos(theta)

    # Create the wrench vector
    F = np.array([0, 0, 0, x, y, z])

    return F

def jacobian_from_config(q1, q2, L1, L2):
    return np.array([
        [1, 0],
        [0, np.cos(q1)],
        [0, np.sin(q1)],
        [0, np.cos(q1)*(L1 + L2*np.cos(q1))],
        [np.cos(q1), 0],
        [np.sin(q1), 0],
    ])

In [4]:
# Initialize the viewer.
try:
    viz.initViewer(open=True)
except ImportError as error:
    print(error)
    sys.exit(0)

try:
    viz.loadViewerModel()
except AttributeError as error:
    print(error)
    sys.exit(0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/


In [12]:
q1 = 0
q2 = 0
L1 = 0
L2 = 0.023
theta = np.pi/2
phi = 0

q = config_to_q([q1, q2])
F = -1 * angle_mag_to_wrench(theta, phi, 75)
viz.display(q)
pin.forwardKinematics(model, data, q)

# J_num = pin.computeJointJacobian(model, data, q, end_effector_joint_id)[:, 6:]
J_num = pin.computeFrameJacobian(model, data, q, end_effector_frame_id)[:, 6:]
J_an = jacobian_from_config(q1, q2, L1, L2)
tau_num = J_num.T @ F
tau_an = J_an.T @ F

print(tau_num)
print(tau_an)

[  0. -75.]
[ 0.    -1.725]
